In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypertension"
cohort = "GSE149256"

# Input paths
in_trait_dir = "../../input/GEO/Hypertension"
in_cohort_dir = "../../input/GEO/Hypertension/GSE149256"

# Output paths
out_data_file = "../../output/preprocess/Hypertension/GSE149256.csv"
out_gene_data_file = "../../output/preprocess/Hypertension/gene_data/GSE149256.csv"
out_clinical_data_file = "../../output/preprocess/Hypertension/clinical_data/GSE149256.csv"
json_path = "../../output/preprocess/Hypertension/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The Association between Poverty and Gene Expression within Immune Cells in a Diverse Baltimore City Cohort"
!Series_summary	"Socioeconomic status (SES), living in poverty, and other social determinants of health contribute to health disparities in the United States. African American (AA) men living below poverty in Baltimore City have a higher incidence of mortality when compared to either white males or AA females living below poverty. Previous studies in our laboratory and elsewhere suggest that environmental conditions are associated with differential gene expression (DGE) patterns in white blood cells, and this may contribute to the onset of diseases in the immune or cardiovascular systems. DGE have also been associated with hypertension and cardiovascular disease (CVD) and correlate with race and gender. However, no studies have investigated how poverty status associates with DGE between male and female AAs and whites living in Baltimore City

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Check Gene Expression Availability
# Based on background info, this is a microarray study on PBMCs examining gene expression profiles
is_gene_available = True  

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# After reviewing the sample characteristics, we don't have direct hypertension data
# The study focuses on poverty status and gene expression, not specifically hypertension
trait_row = None  # No direct hypertension data in this dataset

age_row = 3  # Age is available with various values
gender_row = 0  # Gender is available

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert trait string to binary (0 for negative, 1 for positive)."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Since we don't have direct hypertension data, this function won't be used
    # but we define it to maintain the code structure
    return None

def convert_age(value: str) -> float:
    """Convert age string to float."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """Convert gender string to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
# Since trait_row is None, we skip this substep
if trait_row is not None:
    # This block won't execute since trait_row is None
    # But we keep it for completeness
    pass


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['ILMN_1343061', 'ILMN_1343291', 'ILMN_1343295', 'ILMN_1343321',
       'ILMN_1343339', 'ILMN_1343553', 'ILMN_1343567', 'ILMN_1343638',
       'ILMN_1343668', 'ILMN_1343782', 'ILMN_1343835', 'ILMN_1343841',
       'ILMN_1343872', 'ILMN_1343914', 'ILMN_1343977', 'ILMN_1344038',
       'ILMN_1344055', 'ILMN_1344056', 'ILMN_1651199', 'ILMN_1651209'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I notice they start with "ILMN_" which indicates they are Illumina 
# BeadArray probe IDs, not standard human gene symbols. These IDs need to be mapped to standard gene symbols
# for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343061', 'ILMN_1343291', 'ILMN_1343295', 'ILMN_1343321', 'ILMN_1343339'], 'ARRAY_ADDRESS_ID': ['2900397', '3450719', '4490161', '5390750', '4780100'], 'TRANSCRIPT': ['ILMN_160461', 'ILMN_137991', 'ILMN_137405', 'ILMN_160027', 'ILMN_160401'], 'ILMN_GENE': ['CY3_HYB:HIGH_1_MM2', 'EEF1A1', 'GAPDH', 'NEGATIVE_0971', 'NEGATIVE_0953'], 'PA_Call': [1.0, 1.0, 1.0, 0.0, 0.0], 'TARGETID': ['CY3_HYB:HIGH_1_MM2', 'EEF1A1', 'GAPDH', 'NEGATIVE_0971', 'NEGATIVE_0953'], 'SPECIES': ['ILMN Controls', 'Homo sapiens', 'Homo sapiens', 'ILMN Controls', 'ILMN Controls'], 'SOURCE': ['ILMN_Controls', 'RefSeq', 'RefSeq', 'ILMN_Controls', 'ILMN_Controls'], 'SEARCH_KEY': ['cy3_hyb:high_1_mm2', 'NM_001402.4', nan, 'negative_0971', 'negative_0953'], 'SOURCE_REFERENCE_ID': ['cy3_hyb:high_1_mm2', 'NM_001402.4', 'NM_002046.2', 'negative_0971', 'negative_0953'], 'REFSEQ_ID': [nan, 'NM_001402.4', 'NM_002046.2', nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENTREZ_GENE_ID': 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. After observing the gene expression data and gene annotation data:
# - The gene expression data index contains identifiers like 'ILMN_1343061'
# - In the gene annotation, 'ID' column has the same identifiers
# - The 'SYMBOL' column contains the gene symbols we need to map to

# 2. Create gene mapping dataframe using get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='SYMBOL')

# Display first few rows of the mapping
print("Preview of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
# The apply_gene_mapping function handles the many-to-many relationship as specified
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Display the first few rows of the converted gene expression data
print("\nPreview of gene expression data after mapping:")
print(gene_data.head())


Preview of gene mapping:
             ID                Gene
0  ILMN_1343061  cy3_hyb:high_1_mm2
1  ILMN_1343291              EEF1A1
2  ILMN_1343295               GAPDH
3  ILMN_1343321       negative_0971
4  ILMN_1343339       negative_0953



Preview of gene expression data after mapping:
       GSM4494979  GSM4494980  GSM4494981  GSM4494982  GSM4494983  GSM4494984  \
Gene                                                                            
A1BG        -0.82       -1.04       -0.95       -0.94       -0.80       -0.71   
A2BP1       -2.19       -2.08       -2.07       -2.37       -2.09       -2.28   
A2LD1        0.15        0.12        0.18       -0.02       -0.01        0.00   
A2M         -0.53       -0.71       -0.50       -0.68       -0.59       -0.51   
A2ML1       -0.58       -0.54       -0.54       -0.60       -0.42       -0.60   

       GSM4494985  GSM4494986  GSM4494987  GSM4494988  ...  GSM4495021  \
Gene                                                   ...               
A1BG        -0.82       -1.02       -0.65       -0.97  ...       -0.91   
A2BP1       -2.30       -2.37       -1.98       -2.41  ...       -2.08   
A2LD1       -0.15       -0.04       -0.34        0.01  ...       -0.07   
A2M         -0

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since trait_row is None (hypertension data is not available), we can't extract clinical features
# for our trait of interest. We'll still save age and gender if available.
if trait_row is None:
    # Create a simple clinical dataframe with just age and gender if available
    feature_list = []
    
    if age_row is not None:
        age_data = get_feature_data(clinical_data, age_row, 'Age', convert_age)
        feature_list.append(age_data)
    if gender_row is not None:
        gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
        feature_list.append(gender_data)
        
    if feature_list:
        selected_clinical_df = pd.concat(feature_list, axis=0)
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data (age and gender only) saved to {out_clinical_data_file}")
    else:
        selected_clinical_df = pd.DataFrame()
        print("No clinical data available to save.")
else:
    # This branch would handle the case if trait data were available
    # but since trait_row is None, this code won't execute
    pass

# 3. Since trait data is not available, we can't perform a trait-gene association
# For validation purposes, create a valid DataFrame with the gene data
df_for_validation = pd.DataFrame(index=normalized_gene_data.columns)
# Add age and gender if available
if 'Age' in locals() and 'age_data' in locals():
    df_for_validation['Age'] = age_data.iloc[0]
if 'Gender' in locals() and 'gender_data' in locals():
    df_for_validation['Gender'] = gender_data.iloc[0]

# Save information about why this dataset isn't usable
note = "Dataset from a study on poverty and gene expression in Baltimore. No hypertension trait information available."
validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False, 
    is_biased=False,  # Set to False as we can't determine bias without trait data
    df=df_for_validation,  # Use a concrete DataFrame for validation
    note=note
)

print("Dataset is not usable for hypertension-gene association studies as it doesn't contain trait information.")

Normalized gene data saved to ../../output/preprocess/Hypertension/gene_data/GSE149256.csv
Clinical data (age and gender only) saved to ../../output/preprocess/Hypertension/clinical_data/GSE149256.csv
Abnormality detected in the cohort: GSE149256. Preprocessing failed.
Dataset is not usable for hypertension-gene association studies as it doesn't contain trait information.
